#  MOVIE RECOMMENDATIN SYSTEM

In [ ]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DATA_SETS/Copy of Copy of movies.csv')

In [ ]:
data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [ ]:
data.describe()

,index,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4803.000000,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2401.000000,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,1386.651002,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,1200.500000,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,2401.000000,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,3601.500000,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,4802.000000,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


In [ ]:
data.shape

(4803, 24)

In [ ]:
data.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [ ]:
data.isnull().sum()

,0
index,0
budget,0
genres,28
homepage,3091
id,0
keywords,412
original_language,0
original_title,0
overview,3
popularity,0


In [ ]:
features = ['genres','keywords','tagline','cast','director']

In [ ]:
features

['genres', 'keywords', 'tagline', 'cast', 'director']

In [ ]:
data[features].isnull().sum()

,0
genres,28
keywords,412
tagline,844
cast,43
director,30


In [ ]:
for features in features:
  data[features] = data[features].fillna('')

In [ ]:
data[features].isnull().sum()

np.int64(0)

In [ ]:
data['combine'] = data['genres'] +  data['keywords'] +  data['tagline'] +  data['cast'] +  data['director']

In [ ]:
data['combine']

,combine
0,Action Adventure Fantasy Science Fictioncultur...
1,Adventure Fantasy Actionocean drug abuse exoti...
2,Action Adventure Crimespy based on novel secre...
3,Action Crime Drama Thrillerdc comics crime fig...
4,Action Adventure Science Fictionbased on novel...
...,...
4798,Action Crime Thrillerunited states\u2013mexico...
4799,Comedy RomanceA newlywed couple's honeymoon is...
4800,Comedy Drama Romance TV Moviedate love at firs...
4801,A New Yorker in ShanghaiDaniel Henney Eliza Co...


In [ ]:
# vectorid

vectorizer = TfidfVectorizer()
film_vector = vectorizer.fit_transform(data['combine'])
print(film_vector)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 111820 stored elements and shape (4803, 27580)>
  Coords	Values
  (0, 245)	0.07858581760804864
  (0, 444)	0.09109355212252296
  (0, 8936)	0.11806131645084655
  (0, 21836)	0.09966592997173947
  (0, 9304)	0.27385789694599244
  (0, 4456)	0.2179967521551067
  (0, 10193)	0.1653243242081299
  (0, 22916)	0.3366875640692919
  (0, 26334)	0.130597230719163
  (0, 4676)	0.24529097704249617
  (0, 22778)	0.27385789694599244
  (0, 24197)	0.07518543993419267
  (0, 27161)	0.12619886579371511
  (0, 18164)	0.08690831799482268
  (0, 18558)	0.26121683351048536
  (0, 21521)	0.15623566659433683
  (0, 27182)	0.23480088356130557
  (0, 27540)	0.19771357974524179
  (0, 21487)	0.21550128478931552
  (0, 22464)	0.2020708956871175
  (0, 26547)	0.19638671147741735
  (0, 23267)	0.16128139780622522
  (0, 14550)	0.22368093636480682
  (0, 16614)	0.1569607387745327
  (0, 20729)	0.27385789694599244
  :	:
  (4801, 18790)	0.22450140828104795
  (4801, 11776)	0.3005

In [ ]:
similarity = cosine_similarity(film_vector)

In [ ]:
print(similarity)

[[1.         0.06865296 0.01492221 ... 0.         0.         0.        ]
 [0.06865296 1.         0.02799128 ... 0.01243107 0.         0.        ]
 [0.01492221 0.02799128 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.01243107 0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [ ]:
from operator import index

def Movie_recommendation():
  movie_name = input('Enter your favourite movie name : ')
  list_of_all_titles = data['title'].tolist()
  find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
  close_match = find_close_match[0]
  index_of_the_movie = data[data.title == close_match]['index'].values[0]
  similarity_score = list(enumerate(similarity[index_of_the_movie]))
  similarity_score_sorted = sorted(similarity_score, key = lambda x:x[1], reverse = True)
  print('Movies suggested for you : \n')
  for i in range(29):
    index = similarity_score_sorted[i][0]
    print(data['title'][index])

Movie_recommendation()

Enter your favourite movie name : iron man
Movies suggested for you : 

Iron Man
Iron Man 2
Iron Man 3
The Avengers
Captain America: Civil War
The Helix... Loaded
Avengers: Age of Ultron
X-Men: The Last Stand
X-Men: Days of Future Past
X-Men
X2
Made
Tropic Thunder
The Incredible Hulk
X-Men: Apocalypse
The Amazing Spider-Man 2
Captain America: The Winter Soldier
Gothika
X-Men: First Class
A Scanner Darkly
The Judge
Deadpool
The Best Man
The Nativity Story
Lucky You
The Kite Runner
Arlington Road
Ant-Man
The Hunting Party


In [ ]:
import pickle

# Save similarity matrix
pickle.dump(similarity, open('similarity.pkl', 'wb'))

# Save the minimal required data: title + index + combine
data[['index', 'title', 'combine','overview','cast']].to_csv('movies.csv', index=False)


In [ ]:
from google.colab import files

# Download movies.csv
files.download('movies.csv')

# Download similarity.pkl
files.download('similarity.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>